In [1]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import sys
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

MODEL_NAME = ".\\{}_{}_10_res_loss_10_power_features.pt".format("Positive","FLX")
EPM_TRAIN_FILE = ".\\EPM_model\\EPM_train_dict_May_17.pkl"
EPM_VAL_FILE = ".\\EPM_model\\EPM_val_dict_May_17.pkl"
OFT_TRAIN_FILE_NAME = ".\\OFT_model\\OFT_train_dict_old_features_hand_picked.pkl"
OFT_VAL_FILE_NAME = ".\\OFT_model\\OFT_validation_dict_old_features_hand_picked.pkl"
FLX_TRAIN_FILE = ".\\FLX_model\\FLX_train_dict_old_features.pkl"
FLX_VAL_FILE = ".\\FLX_model\\FLX_validation_dict_old_features.pkl"
FLX_FULL_DATA_TRAIN = ".\\FLX_model\\FLX_train_dict_full_recordings.pkl"
FLX_FULL_DATA_VAL = ".\\FLX_model\\FLX_val_dict_full_recordings.pkl"
FEATURE_WEIGHT = [10,1,1]

FEATURE_LIST = ['X_power_1_2','X_coh_1_2','X_gc_1_2']
FEATURE_VECTOR = FEATURE_LIST
FEATURE_WEIGHTS = [10,1,1]
NETWORK_CONSTRAINT = "Positive"
UMC_PATH = ".\\Universal-Mouse-Code\\"

sys.path.append(UMC_PATH)
#from dCSFA_model import dCSFA_model
import umc_data_tools as umc_dt
from dCSFA_NMF import dCSFA_NMF

if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"


print("Using device: %s"%(device))

#For Consistency
RANDOM_STATE=42

import pandas as pd

model = torch.load(MODEL_NAME,map_location='cpu')
model.device = "cpu"

Using device: cuda:0


In [2]:
with open(FLX_TRAIN_FILE,"rb") as f:
    flx_train_dict = pickle.load(f)

with open(FLX_VAL_FILE,"rb") as f:
    flx_validation_dict = pickle.load(f)
    
#scale_vector = np.array([np.arange(1,57) for feature in range(64)]).flatten()
#Load the data
running_idx = 0
feature_groups = []
for idx,feature in enumerate(FEATURE_LIST):
    f_begin = running_idx
    f_end = f_begin + flx_train_dict[feature].shape[1] 
    if idx == 0:
        f_end = f_end -1
    feature_groups.append((f_begin,f_end))

    running_idx = f_end

flx_X_train = np.hstack([flx_train_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
flx_y_train = flx_train_dict['y_flx']
flx_y_mouse_train = flx_train_dict['y_mouse']
flx_y_expDate_train = flx_train_dict['y_expDate']
flx_y_time_train = flx_train_dict['y_time']

flx_X_validation = np.hstack([flx_validation_dict[feature]*weight for feature,weight in zip(FEATURE_LIST,FEATURE_WEIGHT)])
flx_y_validation = flx_validation_dict['y_flx']
flx_y_mouse_validation = flx_validation_dict['y_mouse']
flx_y_expDate_validation = flx_validation_dict['y_expDate']
flx_y_time_validation = flx_validation_dict['y_time']

flx_X = np.vstack([flx_X_train,flx_X_validation])
flx_y_task = np.hstack([flx_y_train,flx_y_validation])
flx_y_mouse = np.hstack([flx_y_mouse_train,flx_y_mouse_validation])
flx_y_expDate = np.hstack([flx_y_expDate_train,flx_y_expDate_validation])
flx_y_time = np.hstack([flx_y_time_train,flx_y_time_validation])
intercept_mask = OneHotEncoder().fit_transform(flx_y_mouse_train.reshape(-1,1)).todense()

In [6]:
FULL_FEATURE_LIST = ["X_power_1_2_full","X_coh_1_2_full","X_gc_1_2_full"]

flx_X_train = np.hstack([flx_train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
flx_y_train = flx_train_dict['y_flx']
flx_y_mouse_train = flx_train_dict['y_mouse']
flx_y_expDate_train = flx_train_dict['y_expDate']
flx_y_time_train = flx_train_dict['y_time']

flx_X_validation = np.hstack([flx_validation_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
flx_y_validation = flx_validation_dict['y_flx']
flx_y_mouse_validation = flx_validation_dict['y_mouse']
flx_y_expDate_validation = flx_validation_dict['y_expDate']
flx_y_time_validation = flx_validation_dict['y_time']

flx_X = np.vstack([flx_X_train,flx_X_validation])
flx_y_task = np.hstack([flx_y_train,flx_y_validation])
flx_y_mouse = np.hstack([flx_y_mouse_train,flx_y_mouse_validation])
flx_y_expDate = np.hstack([flx_y_expDate_train,flx_y_expDate_validation])
flx_y_time = np.hstack([flx_y_time_train,flx_y_time_validation])

y_pred, s = model.predict(flx_X_train,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_train,flx_y_mouse_train)
print("train roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_train,flx_y_mouse_train,s,True)
print("train mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))
y_pred, s = model.predict(flx_X_validation,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_validation,flx_y_mouse_validation)
print("val roc auc mean: %0.2f +/- %0.2f"%(roc_mean,roc_std))
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,flx_y_validation,flx_y_mouse_validation,s,True)
print("val mw auc mean: %0.2f +/- %0.2f"%(mw_mean,mw_std))

y_pred, s = model.predict(flx_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse)
print("ROC AUCs: ",roc_auc_dict)
mw_auc_dict = umc_dt.lpne_auc(y_pred,flx_y_task,flx_y_mouse,s,True)
print("MW AUCs: ",mw_auc_dict)

df = umc_dt.make_projection_csv(FLX_TRAIN_FILE,model,FULL_FEATURE_LIST,
                    ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                    save_file = ".\\July_30_Projections\\{}_FLX_onto_FLX_flx_vs_saline_Train_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict,mousevar="y_mouse_full")

df = umc_dt.make_projection_csv(FLX_VAL_FILE,model,FULL_FEATURE_LIST,
                    ['y_mouse_full','y_expDate_full','y_time_full','y_flx_full'],
                    save_file = ".\\July_30_Projections\\{}_FLX_onto_FLX_flx_vs_saline_Val_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict,mousevar="y_mouse_full")

c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


train roc auc mean: 0.84 +/- 0.03
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
train mw auc mean: 1.00 +/- 0.00


g:\Anx-Analyses-March-7\Final_Analyses\.\Universal-Mouse-Code\umc_data_tools.py:277: RuntimeWarning: invalid value encountered in double_scalars
  std = np.std(auc_list) / np.sqrt(len(auc_list)-1)
g:\Anx-Analyses-March-7\Final_Analyses\.\Universal-Mouse-Code\umc_data_tools.py:273: RuntimeWarning: invalid value encountered in double_scalars
  std = np.std(auc_list) / np.sqrt(len(auc_list)-1)


val roc auc mean: 0.59 +/- nan
val mw auc mean: 0.66 +/- nan


c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


ROC AUCs:  {'auc_method': 'sklearn_roc_auc', 'Mouse3191': 0.9026334661847197, 'Mouse3192': 0.5243080072570364, 'Mouse3193': 0.8513127198507965, 'Mouse3194': 0.9415610916778505, 'Mouse3202': nan, 'Mouse3203': 0.7906219303097431, 'Mouse99003': nan}
Mouse  Mouse3202  has only one class - AUC cannot be calculated
n_positive samples  1504
n_negative samples  0
Mouse  Mouse99003  has only one class - AUC cannot be calculated
n_positive samples  1874
n_negative samples  0
MW AUCs:  {'auc_method': 'mannWhitneyU', 'Mouse3191': (0.9978038011315429, 0.0), 'Mouse3192': (0.6530373516720605, 1.1666947956043844e-56), 'Mouse3193': (0.9988499463604681, 0.0), 'Mouse3194': (0.9990340799768633, 0.0), 'Mouse3203': (0.9982608647042802, 0.0)}


## EPM Projections

In [3]:
with open(EPM_TRAIN_FILE,"rb") as f:
    epm_train_dict = pickle.load(f)

with open(EPM_VAL_FILE,"rb") as f:
    epm_validation_dict = pickle.load(f)
#Load the data
NUM_FREQS = 56
NUM_FEATURES = (epm_train_dict["X_power_1_2"].shape[1] + \
                epm_train_dict["X_coh_1_2"].shape[1] + \
                epm_train_dict["X_gc_1_2"].shape[1]) // NUM_FREQS
SCALE_VECTOR = np.array([np.arange(1,57) for feature in range(NUM_FEATURES)]).flatten()

X_train = np.hstack([epm_train_dict["X_power_1_2"]*FEATURE_WEIGHTS[0],
                    epm_train_dict["X_coh_1_2"],
                    epm_train_dict["X_gc_1_2"]])#*SCALE_VECTOR

y_train = (epm_train_dict['y_ROI']%2).astype(bool)
y_in_task_mask_train = ~epm_train_dict['y_Homecage'].astype(bool)
y_mouse_train = epm_train_dict['y_mouse']
y_time_train = epm_train_dict['y_time']
train_nan_mask = (epm_train_dict['y_ROI'] > 0)


X_train_task = X_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_train_task = y_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_mouse_train_task = y_mouse_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]
y_time_train_task = y_time_train[np.logical_and(y_in_task_mask_train==1,train_nan_mask)==1]

X_val = np.hstack([epm_validation_dict["X_power_1_2"]*FEATURE_WEIGHTS[0],
                    epm_validation_dict["X_coh_1_2"],
                    epm_validation_dict["X_gc_1_2"]])#*SCALE_VECTOR

y_val = (epm_validation_dict['y_ROI']%2).astype(bool)
y_in_task_mask_val= ~epm_validation_dict['y_Homecage'].astype(bool)
y_mouse_val = epm_validation_dict['y_mouse']
y_time_val = epm_validation_dict['y_time']
val_nan_mask = (epm_validation_dict['y_ROI'] > 0)

X_val_task = X_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_val_task = y_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_mouse_val_task = y_mouse_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]
y_time_val_task = y_time_val[np.logical_and(y_in_task_mask_val==1,val_nan_mask)==1]

epm_X = np.vstack([X_train,X_val])
epm_y_task = np.hstack([y_in_task_mask_train,y_in_task_mask_val])
epm_y_mouse = np.hstack([y_mouse_train,y_mouse_val])
epm_y_time = np.hstack([y_time_train,y_time_val])

y_pred, s = model.predict(epm_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,epm_y_task,epm_y_mouse)
mw_auc_dict = umc_dt.lpne_auc(y_pred,epm_y_task,epm_y_mouse,s,True)
print("ROC AUC dict: ",roc_auc_dict)
print("MW auc dict: ",mw_auc_dict)

df = umc_dt.make_projection_csv(EPM_TRAIN_FILE,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                    save_file = ".\\July_30_Projections\\{}_FLX_onto_EPM_HC_vs_Task_Train_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict)

df = umc_dt.make_projection_csv(EPM_VAL_FILE,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel','y_locx','y_locy'],
                    save_file = ".\\July_30_Projections\\{}_FLX_onto_EPM_HC_vs_Task_Val_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict)

y_pred, s = model.predict(epm_X,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,epm_y_task,epm_y_mouse)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,epm_y_task,epm_y_mouse,s,True)
print("FLX->EPM roc-auc: {} +/- {}".format(roc_mean,roc_std))
print("FLX->EPM mw-auc: {:0.2f} +/- {:0.2f}".format(mw_mean,mw_std))
#umc_dt.make_recon_plots(model,flx_X,0,task="FLX onto EPM (HC vs Task) Train and Val",saveFile="flx_onto_epm_recon.png")

ROC AUC dict:  {'auc_method': 'sklearn_roc_auc', 'Mouse04193': 0.4999170876770178, 'Mouse04201': 0.5083145810290248, 'Mouse04202': 0.5059289442358759, 'Mouse04205': 0.5, 'Mouse04215': 0.4890957845784578, 'Mouse0630': 0.43684095247035265, 'Mouse0633': 0.5457186718938327, 'Mouse0634': 0.5046213093709884, 'Mouse0642': 0.5055807646169092, 'Mouse0643': 0.41035502958579884, 'Mouse1551': 0.4690484194275327, 'Mouse39114': 0.4693084693084693, 'Mouse39124': 0.5310246231847033, 'Mouse39125': 0.5082842013320231, 'Mouse39133': 0.46297807206818425, 'Mouse6291': 0.4088313171694097, 'Mouse6292': 0.48170855461218415, 'Mouse6293': 0.4736559139784946, 'Mouse69064': 0.477074939564867, 'Mouse69065': 0.5163890739506996, 'Mouse69074': 0.5029383267335075, 'Mouse8580': 0.4160910059456302, 'Mouse8581': 0.4684172335600907, 'Mouse8582': 0.5714285714285714, 'Mouse8891': 0.5443403876840976, 'Mouse8894': 0.3804690396737115}
MW auc dict:  {'auc_method': 'mannWhitneyU', 'Mouse04193': (0.48169848657373115, 0.4375937268

## OFT Projections


In [4]:
with open(OFT_TRAIN_FILE_NAME,'rb') as f:
    train_dict = pickle.load(f)

with open(OFT_VAL_FILE_NAME,'rb') as f:
    val_dict = pickle.load(f)

NUM_FREQS = 56
NUM_FEATURES = np.hstack([train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)]).shape[1] // NUM_FREQS

#Train Arrays
oft_X_train = np.hstack([train_dict[feature]*weight for feature,weight in zip(FEATURE_VECTOR,FEATURE_WEIGHTS)])
oft_y_hc_train = train_dict['y_Homecage'].astype(bool)
oft_y_task_train = ~oft_y_hc_train
oft_y_ROI_train = train_dict['y_ROI']
oft_y_vel_train = train_dict['y_vel']
oft_y_mouse_train = train_dict['y_mouse']
oft_y_time_train = train_dict['y_time']

#Validation Arrays
oft_X_val = np.hstack([val_dict[feature] for feature in FEATURE_VECTOR])
oft_y_hc_val = val_dict['y_Homecage'].astype(bool)
oft_y_task_val = ~oft_y_hc_val
oft_y_ROI_val = val_dict['y_ROI']
oft_y_vel_val = val_dict['y_vel']
oft_y_mouse_val = val_dict['y_mouse']
oft_y_time_val = val_dict['y_time']

oft_X = np.vstack([oft_X_train,oft_X_val])
oft_y_task = np.hstack([oft_y_task_train,oft_y_task_val])
oft_y_mouse = np.hstack([oft_y_mouse_train,oft_y_mouse_val])

y_pred, s = model.predict(oft_X,include_scores=True)
roc_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task,oft_y_mouse)
mw_auc_dict = umc_dt.lpne_auc(y_pred,oft_y_task,oft_y_mouse,s,True)
print("ROC AUC dict: ",roc_auc_dict)
print("MW AUC dict: ",mw_auc_dict)

df = umc_dt.make_projection_csv(OFT_TRAIN_FILE_NAME,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = ".\\July_30_Projections\\FLX_onto_OFT_HC_vs_Task_Train_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict)

df = umc_dt.make_projection_csv(OFT_VAL_FILE_NAME,model,FEATURE_VECTOR,
                    ['y_mouse','y_expDate','y_time','y_Homecage','y_ROI','y_vel'],
                    save_file = ".\\July_30_Projections\\FLX_onto_OFT_HC_vs_Task_Val_Projection_2022_July_30.csv".format(NETWORK_CONSTRAINT),
                    auc_dict=mw_auc_dict)

y_pred, s = model.predict(oft_X,include_scores=True)
roc_mean,roc_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task,oft_y_mouse)
mw_mean,mw_std = umc_dt.get_mean_std_err_auc(y_pred,oft_y_task,oft_y_mouse,s,True)
print("EPM->FLX roc-auc: {} +/- {}".format(roc_mean,roc_std))
print("FLX->OFT mw-auc: {:0.2f} +/- {:0.2f}".format(mw_mean,mw_std))
#umc_dt.make_recon_plots(model,oft_X,0,task="FLX onto OFT (HC vs Task) Train and Val",saveFile="flx_onto_oft_recon.png")

c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


Mouse  Mouse04215  has only one class - AUC cannot be calculated
n_positive samples  506
n_negative samples  0
Mouse  Mouse3192  has only one class - AUC cannot be calculated
n_positive samples  279
n_negative samples  0
Mouse  Mouse3194  has only one class - AUC cannot be calculated
n_positive samples  324
n_negative samples  0
Mouse  Mouse3203  has only one class - AUC cannot be calculated
n_positive samples  500
n_negative samples  0
ROC AUC dict:  {'auc_method': 'sklearn_roc_auc', 'Mouse04191': 0.3643413771034898, 'Mouse04193': 0.47769328263624844, 'Mouse04201': 0.5343522521005258, 'Mouse04202': 0.5543531276673577, 'Mouse04205': 0.5172682350413579, 'Mouse04215': nan, 'Mouse3191': 0.40757380532573806, 'Mouse3192': nan, 'Mouse3193': 0.47506395906619764, 'Mouse3194': nan, 'Mouse3203': nan, 'Mouse39114': 0.5008647798742139, 'Mouse39124': 0.4566960252935862, 'Mouse39125': 0.4322689888727625, 'Mouse39133': 0.2969087238042191, 'Mouse69064': 0.4940508593203204, 'Mouse69065': 0.403654827848

c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
c:\Users\micha\Anaconda3\envs\cpne\lib\site-packages\sklearn\metrics\_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
